In [5]:
import wfdb
import matplotlib.pyplot as plt
import numpy as np
# from hrv.filters import quotient, moving_median
from scipy import interpolate
from tqdm import tqdm
import os
FS = 100.0

In [7]:
def create_time_info(rri):
    rri_time = np.cumsum(rri) / 1000.0  # make it seconds
    return rri_time - rri_time[0]   # force it to start at zero

def create_interp_time(rri, fs):
    time_rri = create_time_info(rri)
    return np.arange(0, time_rri[-1], 1 / float(fs))

def interp_cubic_spline(rri, fs):
    time_rri = create_time_info(rri)
    time_rri_interp = create_interp_time(rri, fs)
    tck = interpolate.splrep(time_rri, rri, s=0)
    rri_interp = interpolate.splev(time_rri_interp, tck, der=0)
    return time_rri_interp, rri_interp

def interp_cubic_spline_qrs(qrs_index, qrs_amp, fs):
    time_qrs = qrs_index / float(FS)
    time_qrs = time_qrs - time_qrs[0]
    time_qrs_interp = np.arange(0, time_qrs[-1], 1/float(fs))
    tck = interpolate.splrep(time_qrs, qrs_amp, s=0)
    qrs_interp = interpolate.splev(time_qrs_interp, tck, der=0)
    return time_qrs_interp, qrs_interp




In [15]:
data_path = '../res/origin/'
train_data_name = ['a01', 'a02', 'a03', 'a04', 'a05',
             'a06', 'a07', 'a08', 'a09', 'a10',
             'a11', 'a12', 'a13', 'a14', 'a15',
             'a16', 'a17', 'a18', 'a19',
             'b01', 'b02', 'b03', 'b04',
             'c01', 'c02', 'c03', 'c04', 'c05',
             'c06', 'c07', 'c08', 'c09',
             ]
test_data_name = ['a20','b05','c10']
age = [51, 38, 54, 52, 58,
       63, 44, 51, 52, 58,
       58, 52, 51, 51, 60,
       44, 40, 52, 55, 58,
       44, 53, 53, 42, 52,
       31, 37, 39, 41, 28,
       28, 30, 42, 37, 27]
sex = [1, 1, 1, 1, 1,
       1, 1, 1, 1, 1,
       1, 1, 1, 1, 1,
       1, 1, 1, 1, 1,
       0, 1, 1, 1, 1,
       1, 1, 1, 0, 0,
       0, 0, 1, 1, 1]


In [11]:
# Return the amplitude (max - min) of the RRi series: biên độ
def get_qrs_amp(ecg, qrs):
    interval = int(FS * 0.250)
    qrs_amp = []
    for index in range(len(qrs)):
        curr_qrs = qrs[index]
        amp = np.max(ecg[curr_qrs-interval:curr_qrs+interval])

        qrs_amp.append(amp)

    return qrs_amp



In [ ]:
MARGIN = 10
FS_INTP = 4
MAX_HR = 300.0
MIN_HR = 20.0
MIN_RRI = 1.0 / (MAX_HR / 60.0) * 1000
MAX_RRI = 1.0 / (MIN_HR / 60.0) * 1000
train_input_array = []
train_label_array = []

for data_index in range(len(train_data_name)):
    print("in for")
    filePath = data_path + train_data_name[data_index]
    print (filePath)
    win_num = len(wfdb.rdann(os.path.join(data_path,train_data_name[data_index]), 'apn').symbol)
    signals, fields = wfdb.rdsamp(os.path.join(data_path,train_data_name[data_index]))
    for index in tqdm(range(1, win_num)):
        samp_from = index * 60 * FS # 60 seconds
        samp_to = samp_from + 60 * FS  # 60 seconds

        # array dữ liệu sạch
        # from -> to: 80 seconds
        qrs_ann = wfdb.rdann(data_path + train_data_name[data_index], 'qrs', sampfrom=samp_from - (MARGIN*100), sampto=samp_to + (MARGIN*100)).sample
        # lấy label của dữ liệu apnea
        # from -> to: 6000đv - 1đv
        apn_ann = wfdb.rdann(data_path + train_data_name[data_index], 'apn', sampfrom=samp_from, sampto=samp_to-1).symbol
        # lấy tín hiệu ecg cao nhất
        qrs_amp = get_qrs_amp(signals, qrs_ann)

        # diff: out[i] = a[i+1] - a[i] -> Tinh khoang RR, rri la chuoi cac gia tri RR
        rri = np.diff(qrs_ann)
        rri_ms = rri.astype('float') / FS * 1000.0


 78%|███████▊  | 378/483 [08:35<06:06,  3.49s/it]  

in for
../res/origin/a01
in for
../res/origin/a02
in for
../res/origin/a03
in for
../res/origin/a04
in for
../res/origin/a05
in for
../res/origin/a06
in for
../res/origin/a07
in for
../res/origin/a08
in for
../res/origin/a09
in for
../res/origin/a10
in for
../res/origin/a11
in for
../res/origin/a12
in for
../res/origin/a13
in for
../res/origin/a14
in for
../res/origin/a15
in for
../res/origin/a16
in for
../res/origin/a17
in for
../res/origin/a18
in for
../res/origin/a19
in for
../res/origin/b01
in for
../res/origin/b02
in for
../res/origin/b03
in for
../res/origin/b04
in for
../res/origin/c01
